In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import rtsvg
import networkx as nx
rt = rtsvg.RACETrack()

df = pl.DataFrame({'fm':'a b c d e f g g h h i i i f e'.split(),
                   'to':'b c d a a g f i i i a b c h h'.split(),
                   'ct':[1,9,2,3,4,1,2,7,3,4,2,3,9,2,3]})
relates  = [('fm','to')]
clusters = {'__cluster_a__':['a','b','c'],
            '__cluster_b__':['d','e']} 
g        = rt.createNetworkXGraph(df, relates)
pos      = nx.spring_layout(g)
_link_   = rt.link(df, relates, pos, draw_labels=True,w=256,h=256)

In [ ]:
#
# collapseGraph()
# - limitations -- only single field from/tos can be used (i.e., no multifield nodes)
# - multirelationships get collapsed down to one relationships
#
def collapseGraph(df, relationships, node_clusters, count_by=None, count_by_set=False, color_by=None):
    # Create the reverse maps
    rev_map = {}
    for k, v in node_clusters.items():
        for v_ in v: rev_map[v_] = k 
    
    # Per relationship
    __dfs__ = []
    for _relates_ in relationships:
        _fm_, _to_ = _relates_[0], _relates_[1]
        all_nodes  = set(df[_fm_]) | set(df[_to_])
        # fill in the reverse map
        for n in all_nodes:
            if n not in rev_map: rev_map[n] = n
        # remap the nodes in the dataframe
        rev_map_fn = lambda x: rev_map[x]
        df_tmp     = df.with_columns(pl.col(_fm_).replace_strict(rev_map).alias('__fm__'),
                                     pl.col(_to_).replace_strict(rev_map).alias('__to__'))
        df_counter = rt.polarsCounter(df_tmp, ['__fm__','__to__'], count_by, count_by_set)
        __dfs__.append(df_counter)

    return pl.concat(__dfs__)

df_collapsed     = collapseGraph(df, relates, clusters)
_link_collapsed_ = rt.link(df_collapsed, [('__fm__','__to__')], pos, draw_labels=True,w=256,h=256)
rt.tile([_link_, _link_collapsed_], 20)

In [ ]:
rt.polarsCounter